In [ ]:
import cv2 as cv
import numpy as np
import mediapipe as mp
import math
mp_face_mesh = mp.solutions.face_mesh

cap = cv.VideoCapture(0)
# cap = cv.VideoCapture('video/source/7/CB_MAA_19042022.mp4')
#indices da iris
LEFT_IRIS = [474, 475, 476, 477]
RIGHT_IRIS = [469, 470, 471, 472]

L_H_LEFT = [33]     # olho direito ponto de referência mais à direita
L_H_RIGHT = [133]   # olho direito ponto de referência mais à esquerda
R_H_LEFT = [362]    # olho esquerdo ponto de referência mais à direita
R_H_RIGHT = [263]   # olho esquerdo ponto de referência mais à esquerda

#Funcao que define distancias euclidianas dos pontos nos olhos
def euclidean_distance(point1, point2):
    x1, y1 =point1.ravel()
    x2, y2 =point2.ravel()
    distance = math.sqrt((x2-x1)**2 + (y2-y1)**2)
    return distance

#Funcao para encontra posicao da iris
def iris_position(iris_center, right_point, left_point):
    center_to_right_dist = euclidean_distance(iris_center, right_point)
    total_distance = euclidean_distance(right_point, left_point)
    ratio = center_to_right_dist/total_distance
    cv.putText(frame, "ratio: " + str(np.round(ratio, 2)), (500, 150), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    iris_position =""
    if ratio <= 2.65:
        iris_position="right"
    elif ratio > 2.7 and ratio <= 3.0:
        iris_position="center"
    else:
        iris_position = "left"
    cv.putText(frame, "iris_position: " + iris_position, (550, 200), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    return iris_position, ratio
    

with mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5, min_tracking_confidence=0.5) as face_mesh:
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv.flip(frame, 1)
        rgb_frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)   #Mediapipe precisa do formato de cores RGB mas o OpenCV usa o BGR
        img_h, img_w = frame.shape[:2]
        results = face_mesh.process(rgb_frame)
        if results.multi_face_landmarks:
            mesh_points=np.array([np.multiply([p.x, p.y], [img_w, img_h]).astype(int) for p in results.multi_face_landmarks[0].landmark])

            #transformar formas quadradas em círculos, função do OpenCV fornece círculos delimitadores com base nos pontos fornecidos.
            #minEnclosingCircle que retorna, o centro (x,y) e o raio dos círculos, os valores de retorno são de ponto flutuante, necessario transformá-los em int.
            (l_cx, l_cy), l_radius = cv.minEnclosingCircle(mesh_points[LEFT_IRIS])
            (r_cx,r_cy), r_radius = cv.minEnclosingCircle(mesh_points[RIGHT_IRIS])

            # transforma pontos centrais em array np
            center_left = np.array([l_cx, l_cy], dtype=np.int32)
            center_right = np.array([r_cx, r_cy], dtype=np.int32)

            #desenhe o círculo com base nos valores de retorno da minEnclosingCircle, através do CIRCLE que desenha a imagem do círculo com base no centro (x, y) e no raio
            cv.circle(frame, center_left, int(l_radius), (255, 0, 255), 1, cv.LINE_AA)
            cv.circle(frame, center_right, int(r_radius), (255, 0, 255), 1, cv.LINE_AA)

            #mostrar pontos nos cantos dos olhos
            cv.circle(frame, mesh_points[R_H_RIGHT][0], 3, (255, 255, 255), -1, cv.LINE_AA)
            cv.circle(frame, mesh_points[R_H_LEFT][0], 3, (0, 255, 255), -1, cv.LINE_AA)


            iris_pos, ratio = iris_position(center_right, mesh_points[R_H_RIGHT], mesh_points[R_H_LEFT][0])

            # print(iris_pos)
        cv.imshow("img", frame)
        key = cv.waitKey(1)
        if key ==ord("q"):
            break
cap.release()
cv.destroyAllWindows()

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import time
import datetime
import utils,math
import csv
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5,
    # เพิ่มจำนวน detect หน้าสูงสุด
    max_num_faces=1,
)

mp_drawing = mp.solutions.drawing_utils

drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
count = 0
seconds = time.time()
cap = cv2.VideoCapture(0)
# cap = cv2.VideoCapture('video/source/2/CD_SOS_20042022.mp4')
# cap = cv2.VideoCapture('video/IMG_0378.mp4')
default = list('-'*5)
ith_sample = 0
ohm_test = 0
status = list('-'*10)
lookdown = list()
countL = 0
countR = 0
newx=0
testx=0
text = ''
# variables 
CEF_COUNTER =0
TOTAL_BLINKS =0
# constants
CLOSED_EYES_FRAME =3
student_header = ['name', 'หันซ้าย (ครั้ง)', 'หันขวา (ครั้ง)', 'x']
student_data = []
# *-------------------------------*

# Left eyes indices 
LEFT_EYE =[ 362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385,384, 398 ]
LEFT_EYEBROW =[ 336, 296, 334, 293, 300, 276, 283, 282, 295, 285 ]

# right eyes indices
RIGHT_EYE=[ 33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161 , 246 ]  
RIGHT_EYEBROW=[ 70, 63, 105, 66, 107, 55, 65, 52, 53, 46 ]
 #----save csv-----
def savecsv(countL,countR):
    student_data = [
        ['userA', countL, countR, 'Chemistry'],
        # ['Sophie', 22, 'Physics', 'Computer Science'],
        # ['John', 24, 'Mathematics', 'Physics'],
        # ['Jane', 30, 'Chemistry', 'Physics']
    ]
    with open('students.csv', 'w') as file:
        writer = csv.writer(file)
        writer.writerow(student_header)

        # Use writerows() not writerow()
        writer.writerows(student_data) 
    
    return 0


# landmark detection function 
def landmarksDetection(img, landmark, draw=False):
    img_height, img_width= img.shape[:2]
    # list[(x,y), (x,y)....]
    mesh_coord = [(int(point.x * img_width), int(point.y * img_height)) for point in landmark]
    if draw :
        [cv.circle(img, p, 2, (0,255,0), -1) for p in mesh_coord]

    # returning the list of tuples for each landmarks 
    return mesh_coord

# Euclaidean distance 
def euclaideanDistance(point, point1):
    x, y = point
    x1, y1 = point1
    distance = math.sqrt((x1 - x)**2 + (y1 - y)**2)
    return distance

def checkTrun(checkOpen):
    if(checkOpen == False):
        return 0
    cv2.putText(image, status[9], (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 2)
    cv2.putText(image, status[8], (20, 100), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 2)
  
    # cv2.putText(image, time.time(), (20, 150), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 2)
    if(status[9] != status[8]):
        print('xxx')
    return 0

# Blinking Ratio
def blinkRatio(img, landmarks, right_indices, left_indices):
    # Right eyes 
    # horizontal line 
    rh_right = landmarks[right_indices[0]]
    rh_left = landmarks[right_indices[8]]
    # vertical line 
    rv_top = landmarks[right_indices[12]]
    rv_bottom = landmarks[right_indices[4]]
    # draw lines on right eyes 
    # cv.line(img, rh_right, rh_left, utils.GREEN, 2)
    # cv.line(img, rv_top, rv_bottom, utils.WHITE, 2)

    # LEFT_EYE 
    # horizontal line 
    lh_right = landmarks[left_indices[0]]
    lh_left = landmarks[left_indices[8]]

    # vertical line 
    lv_top = landmarks[left_indices[12]]
    lv_bottom = landmarks[left_indices[4]]

    rhDistance = euclaideanDistance(rh_right, rh_left)
    rvDistance = euclaideanDistance(rv_top, rv_bottom)

    lvDistance = euclaideanDistance(lv_top, lv_bottom)
    lhDistance = euclaideanDistance(lh_right, lh_left)

    reRatio = rhDistance/rvDistance
    leRatio = lhDistance/lvDistance

    ratio = (reRatio+leRatio)/2
    return ratio 
start_time = time.time()
while cap.isOpened():
    success, image = cap.read()
    ith_sample += 1
    start = time.time()

    # Flip the image horizontally for a later selfie-view display
    # Also convert the color space from BGR to RGB
    # image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # To improve performance
    image.flags.writeable = False

    # Get the result
    results = face_mesh.process(image)

    # To improve performance
    image.flags.writeable = True

    # Convert the color space from RGB to BGR
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    img_h, img_w, img_c = image.shape
    face_3d = []
    face_2d = []

    # จำนวนคน
    # print('len(results.multi_face_landmarks)')
    # print(len(results.multi_face_landmarks))
    
    cv2.putText(image, str(ith_sample), (20, 500), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 2)
    cv2.putText(image, ' '.join(status), (20, 550), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 2)
    
    if results.multi_face_landmarks:
        currentDate = str(datetime.datetime.now())
        for i,face_landmarks in enumerate(results.multi_face_landmarks):
            
            # เพิ่มกรอบหน้าขาว
            mp_drawing.draw_landmarks(
                image=image,
                landmark_list=face_landmarks,
                connections=mp_face_mesh.FACEMESH_CONTOURS,
                landmark_drawing_spec=drawing_spec,
                connection_drawing_spec=drawing_spec)
            
            # ส่วนของการกระพริบตา
            mesh_coords = landmarksDetection(image, face_landmarks.landmark, False)
            ratio = blinkRatio(image, mesh_coords, RIGHT_EYE, LEFT_EYE)
            cv2.putText(image, f'Ratio : {round(ratio,2)}', (20, 200), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            if ratio >4.5:
                CEF_COUNTER +=1
                # cv.putText(frame, 'Blink', (200, 50), FONTS, 1.3, utils.PINK, 2)
                utils.colorBackgroundText(image,  f'Blink', cv2.FONT_HERSHEY_SIMPLEX, 0.7, (20, 100), 2, utils.YELLOW, pad_x=6, pad_y=6, )

            else:
                if CEF_COUNTER>CLOSED_EYES_FRAME:
                    TOTAL_BLINKS +=1
                    CEF_COUNTER =0
            # cv.putText(frame, f'Total Blinks: {TOTAL_BLINKS}', (100, 150), FONTS, 0.6, utils.GREEN, 2)
            utils.colorBackgroundText(image,  f'Total Blinks: {TOTAL_BLINKS}', cv2.FONT_HERSHEY_SIMPLEX, 0.7, (20,150),2)

            
            
            
            
            face_2d = []
            face_3d = []
            for idx, lm in enumerate(face_landmarks.landmark):
                # เอาไว้หาตำแหน่งจุดของหน้า
                # test_x, test_y = int(lm.x * img_w), int(lm.y * img_h)
                # cv2.circle(image, (test_x, test_y), 2, (255, 0, 0), -1)
                # cv2.putText(image, str(idx), (test_x, test_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
                
                # เก็บตำแหน่งบนหน้าผาก
                if idx == 10:
                    header_x,header_y = int(lm.x * img_w), (int(lm.y * img_h))
                    
                # เก็บตำแหน่งจมูก
                if idx == 33 or idx == 263 or idx == 1 or idx == 61 or idx == 291 or idx == 199:
                    # print('mosss',i)
                    if idx == 1:
                        nose_2d = (lm.x * img_w, lm.y * img_h)
                        nose_3d = (lm.x * img_w, lm.y * img_h, lm.z * 3000)

                    x, y = int(lm.x * img_w), int(lm.y * img_h)

                    # Get the 2D Coordinates
                    face_2d.append([x, y])

                    # Get the 3D Coordinates
                    face_3d.append([x, y, lm.z])

            # Convert it to the NumPy array
            face_2d = np.array(face_2d, dtype=np.float64)

            # Convert it to the NumPy array
            face_3d = np.array(face_3d, dtype=np.float64)

            # The camera matrix
            focal_length = 1 * img_w

            cam_matrix = np.array([[focal_length, 0, img_h / 2],
                                   [0, focal_length, img_w / 2],
                                   [0, 0, 1]])

            # The distortion parameters
            dist_matrix = np.zeros((4, 1), dtype=np.float64)

            # Solve PnP
            success, rot_vec, trans_vec = cv2.solvePnP(face_3d, face_2d, cam_matrix, dist_matrix)

            # Get rotational matrix
            rmat, jac = cv2.Rodrigues(rot_vec)

            # Get angles
            angles, mtxR, mtxQ, Qx, Qy, Qz = cv2.RQDecomp3x3(rmat)

            # Get the y rotation degree
            x = angles[0] * 360
            y = angles[1] * 360
            z = angles[2] * 360
            
            start_point = (5, 220)
            end_point = (440, 440)
            color = (255, 0, 0)
            
            sec = time.time() - start_time
            
            if(sec <= 10):
                testx = x
                default.append(np.round(x, 2))
                default.pop(0)            
            elif(sec > 10):
                default = default
                
            cv2.putText(image, "default: " + str(default), (0, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            
            if(testx>x):
                newx = testx + x
            else:
                newx = x - testx
            # See where the user's head tilting
            if y < -7.5:
                text = "Looking Right"
                str_status = "R"
                count = count+1
                if(status[9] != status[8]):
                    countR += 1
                    print(countR)        
            elif y > 7:
                text = "Looking Left"
                str_status = "L"
                count = count+1
               
                secL = time.time() 
                print(secL)
                if(status[9] != status[8]):
                    countL += 1
                    print(status[9])
                    print(status[8])
                    
                    
                    print(countL)
            elif newx < -8:
                text = "Looking Down"
                str_status = "D"
                lookdown.append(str_status)
                # print(lookdown)
                count = count+1
            elif newx > 8:
                text = "Looking Up"
                str_status = "U"
            else:
                text = "Forward"
                count = count-1
                str_status = "-"
            
            # update status
            if ith_sample % 1 == 0:
                checkTrun(False)
                status.append(str_status)
                status.pop(0)       
            #---------ohm-------------    
            # print(status[10])
            # ตรวจจับการเข้าข่ายการโกง 
            # if count > 70:
            #     test = cv2.rectangle(image, start_point, end_point, color, thickness)
            if count <= 0:
                count = 0
              
            # Display the nose direction
            nose_3d_projection, jacobian = cv2.projectPoints(nose_3d, rot_vec, trans_vec, cam_matrix, dist_matrix)

            p1 = (int(nose_2d[0]), int(nose_2d[1]))
            p2 = (int(nose_2d[0] + y * 10), int(nose_2d[1] - x * 10))
            
            cv2.line(image, p1, p2, (255, 0, 0), 3)
            
            # if sec >= 5 and sec <= 10:
            #     print('sec')
            # Add the text on the image
            
            cv2.putText(image, text, (header_x,header_y), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
            # cv2.putText(image, "str_status: " +str_status, (header_x,header_y-100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.putText(image, "x: " + str(np.round(newx, 2)), (header_x,header_y-100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.putText(image, "y: " + str(np.round(y, 2)), (header_x,header_y-80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            # cv2.putText(image, "z: " + str(np.round(z, 2)), (header_x,header_y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            # cv2.putText(image,currentDate,(header_x,header_y-40),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,0,255),cv2.LINE_4)
            cv2.putText(image, "seconds: " + str(sec), (header_x,header_y-25), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            
            # cv2.putText(image, ' '.join(status), (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 2)
           
            # cv2.putText(image, "x: " + str(np.round(x, 2)), (500, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            # cv2.putText(image, "y: " + str(np.round(y, 2)), (500, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            # cv2.putText(image, "z: " + str(np.round(z, 2)), (500, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            # cv2.putText(image,currentDate,(500,200),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,0,255),cv2.LINE_4)
            cv2.putText(image, "count: " + str(count), (500, 250), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            # print(time.time() /1000)
        end = time.time()
        totalTime = end - start
        try:
            fps = 1 / totalTime
        except:
            print("An exception occurred")    
        # cv2.putText(image, f'FPS: {int(fps)}', (20, 450), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 2)

    cv2.imshow('Head Pose Estimation', image)

    if cv2.waitKey(5) & 0xFF == 27:
        
        savecsv(countL,countR)
        
        break

cv2.destroyAllWindows()
cap.release()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


1660817974.746251
1660817974.809772
L
-
1
1660817974.84273
1660817974.913763
1660817974.976678
1660817975.039139
1660817978.80934
1660817978.871851
L
-
2
1660817978.942688
1660817979.006902
1660817979.0640042
1660817979.097922
1660817979.132902
1660817979.167711
1660817979.2385201
1660817979.295747
1660817979.329896
1660817979.36496
1660817979.436097
1660817979.5055518
1660817979.563061
1660817979.597128
1660817979.668073
1660817979.7310941
1660817979.763784
1660817979.827976
1660817979.8983512
1660817979.961303
1660817980.0316858
1660817980.09481
1660817980.158697
1660817980.228927
1660817980.292587
1660817980.351169
1660817980.3836858
1660817980.418504
1660817980.4528031
1660817980.524096
1660817980.5872488
1660817980.658615
1660817980.715456
1660817980.7496462
1660817980.7843
1660817980.855754
1660817980.918903
1660817980.981622
1660817981.051827
1660817981.1152499
1660817981.1794
1660817981.212476
1660817981.277524
1660817981.311266
1660817981.375669
1660817981.409693
1660817981.48

In [1]:
#----save vdo-----
import cv2 as cv
cap = cv.VideoCapture(0)
# Define the codec and create VideoWriter object
width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH) + 0.5)
height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT) + 0.5)
size = (width, height)

fourcc = cv.VideoWriter_fourcc(*'XVID')
out = cv.VideoWriter('test.mp4', fourcc, 20.0, size)
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    # frame = cv.flip(frame, 0)
    # write the flipped frame
    out.write(frame)
    cv.imshow('frame', frame)
    if cv.waitKey(1) == ord('q'):
        break
# Release everything if job is finished
cap.release()
out.release()
cv.destroyAllWindows()

OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


In [5]:
import csv

student_header = ['name', 'หันซ้าย', 'หันขวา', 'x']

student_data = [
    ['userA', 23, 0, 'Chemistry'],
    # ['Sophie', 22, 'Physics', 'Computer Science'],
    # ['John', 24, 'Mathematics', 'Physics'],
    # ['Jane', 30, 'Chemistry', 'Physics']
]

with open('students.csv', 'w') as file:
    writer = csv.writer(file)
    writer.writerow(student_header)

    # Use writerows() not writerow()
    writer.writerows(student_data)